In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
from torchvision.transforms.functional import pad as PAD
import torch.optim as optim
from torch.utils.data.dataset import random_split
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
import re
import pandas as pd
from sklearn.metrics import classification_report
import wandb
import numbers
os.environ['CUDA_VISIBLE_DEVICES']='1'

from efficientnet_pytorch import EfficientNet

PyTorch Version:  1.4.0


In [2]:
hyperparameter_defaults = dict(
    input_size = 1024,
    batch_size = 1,
)
os.environ['WANDB_NOTEBOOK_NAME']='Testing Notebook'
wandb.init(project="AnimalRadTesting",config=hyperparameter_defaults)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: szho6430 (use `wandb login --relogin` to force relogin)


In [3]:
wbconfig = wandb.config

In [4]:
input_size = wbconfig.input_size
batch_size = wbconfig.batch_size
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
meta = pd.read_csv('./To be added with class score.csv',dtype={'Image No.':str})
# meta['Img Name'] = meta['Image No.']+'.jpg'
# meta['Direct'] = np.nan
# meta['MURA'] = np.nan
meta

,Index,Patient No.,Image No.,Species,Thoracic or Pelvic,Is_Normal,Image Finding,Direct Correct,Direct_Abn_Score,Direct_Nor_Score,MURA Correct,MURA_Abn_Score,MURA_Nor_Score
0,1,1,001,Canine,Thoracic,Abnormal,Osteochondrosis,1,1.000,0.000,NaN,NaN,NaN
1,2,1,002,Canine,Thoracic,Abnormal,Osteochondrosis,1,0.651,0.349,NaN,NaN,NaN
2,3,1,003,Canine,Thoracic,Abnormal,Osteochondrosis,0,0.329,0.671,NaN,NaN,NaN
3,4,1,004,Canine,Thoracic,Abnormal,Osteochondrosis,1,0.991,0.009,NaN,NaN,NaN
4,5,1,005,Canine,Thoracic,Abnormal,Osteochondrosis,1,1.000,0.000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,140,496,Feline,Thoracic,Abnormal,Mixed Lesion,1,1.000,0.000,NaN,NaN,NaN
496,497,140,497,Feline,Thoracic,Abnormal,Mixed Lesion,0,0.095,0.905,NaN,NaN,NaN
497,498,141,498,Feline,Pelvic,Normal,Normal,0,0.513,0.487,NaN,NaN,NaN
498,499,141,499,Feline,Pelvic,Abnormal,DJD,0,0.013,0.987,NaN,NaN,NaN


In [6]:
meta.groupby(meta['Is_Normal']).count()

,Index,Patient No.,Image No.,Species,Thoracic or Pelvic,Image Finding,Direct Correct,Direct_Abn_Score,Direct_Nor_Score,MURA Correct,MURA_Abn_Score,MURA_Nor_Score
Is_Normal,,,,,,,,,,,,
Abnormal,356,356,356,356,356,356,356,356,356,0,0,0
Normal,144,144,144,144,144,144,144,144,144,0,0,0


In [7]:
def get_padding(image):    
    w, h = image.size
    max_wh = np.max([w, h])
    h_padding = (max_wh - w) / 2
    v_padding = (max_wh - h) / 2
    l_pad = h_padding if h_padding % 1 == 0 else h_padding+0.5
    t_pad = v_padding if v_padding % 1 == 0 else v_padding+0.5
    r_pad = h_padding if h_padding % 1 == 0 else h_padding-0.5
    b_pad = v_padding if v_padding % 1 == 0 else v_padding-0.5
    padding = (int(l_pad), int(t_pad), int(r_pad), int(b_pad))
    return padding

class NewPad(object):
    def __init__(self, fill=0, padding_mode='edge'):
        assert isinstance(fill, (numbers.Number, str, tuple))
        assert padding_mode in ['constant', 'edge', 'reflect', 'symmetric']

        self.fill = fill
        self.padding_mode = padding_mode
        
    def __call__(self, img):
        """
        Args:
            img (PIL Image): Image to be padded.

        Returns:
            PIL Image: Padded image.
        """
        return PAD(img, get_padding(img), self.fill, self.padding_mode)
    
    def __repr__(self):
        return self.__class__.__name__ + '(padding={0}, fill={1}, padding_mode={2})'.\
            format(self.fill, self.padding_mode)
from torchvision.transforms import ToPILImage
ToPILImage = ToPILImage()


def invTrans(image):
    transformation = transforms.Compose([transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.1402, 1/0.1402, 1/0.1402 ]),
                                         transforms.Normalize(mean = [ -0.1523, -0.1523, -0.1523 ],
                                                     std = [ 1., 1., 1. ]),
                               ])
    image = transformation(image)
    return image

def test(metadata,col_name,model,dataloaders, device):
    Softmax = nn.Softmax(dim=0)
    idx=0
    labels = []
    preds = []
    total = 0
    num_correct = 0
    for _,param in model.named_parameters():
        param.requires_grad = False
    model.to(device)
    model.eval()
    for inputs,label in dataloaders['test']:
        idx+=1
        img = invTrans(inputs.squeeze())
        img = ToPILImage(img)
        raw_name = dataloaders_dict['test'].dataset.samples[idx-1][0].split("/")[-2:]
        wandb_name = raw_name[1]
        img_name = raw_name[1]
        img_name = img_name.replace('.jpg','').replace('.jpeg','')
        inputs = inputs.to(device)
        total += 1
        outputs = model(inputs)
        _,pred = torch.max(outputs,1)
        Class_scores = np.array(Softmax(outputs.to('cpu').squeeze()))
        Abn_Score = Class_scores[0]
        Nor_Score = Class_scores[1]
        pred = pred.to('cpu').detach()
#         print("\n\nImage number is {}\tlabel is {}\tOutput is {}\nClass Probability is {}\tprediction is {}\tcorrectness is {}".format(img_name,label,
#                                                                                                          outputs,
#                                                                                                          Class_scores,
#                                                                                                          pred,
#                                                                                                          torch.equal(pred,label.data)))
        metadata.loc[metadata['Image No.']==img_name,'MURA_Abn_Score'] = round(Abn_Score,ndigits=3)
        metadata.loc[metadata['Image No.']==img_name,'MURA_Nor_Score'] = round(Nor_Score,ndigits=3)
        if (torch.equal(pred,label.data))==True:
            metadata.loc[metadata['Image No.']==img_name,'MURA Correct']=str(1)
#             caption = col_name+'_'+raw_name[0]+'_Correctly Classified'
#             wandb.log({wandb_name:[wandb.Image(img,caption=caption)]})
        if (torch.equal(pred,label.data))==False:
            metadata.loc[metadata['Image No.']==img_name,'MURA Correct']=str(0)
#             caption = col_name+'_'+raw_name[0]+'_Incorrect Classified'
#             wandb.log({wandb_name:[wandb.Image(img,caption=caption)]})
        labels.append(int(label.detach()))
        preds.append(int(pred))
    return labels,preds  
    
normalizations = {}
normalizations['MURA'] = [0.1523, 0.1523, 0.1523], [0.1402, 0.1402, 0.1402]
normalizations['Direct'] = [0.1523, 0.1523, 0.1523], [0.1402, 0.1402, 0.1402]
data_transforms = {
    'test': transforms.Compose([
        NewPad(),
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize(normalizations['MURA'][0],normalizations['MURA'][1])
    ]),
}



In [8]:
results={}
results['Direct']= {}
results['MURA']= {}
cv_dir = "/media/dasleo/LEOHDD/data/AnimalDatasetV2CVs/"
for strategy in ['MURA']:
    for epoch in range(99,100):
        results[strategy][epoch] = dict(labels=[],preds=[])
        for cv in np.arange(1,5+1):
            '''Step 1: Build up dataset and dataloader'''
            data_dir = cv_dir+'CV'+str(cv)
            image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), 
                                                      data_transforms[x]) for x in ['test']}
            dataloaders_dict = {}
            dataloaders_dict['test'] = torch.utils.data.DataLoader(image_datasets['test'],
                                                               batch_size=1,
                                                               shuffle=False,
                                                               num_workers=2)
            '''Step 2: Set up model'''
            test_model = EfficientNet.from_pretrained('efficientnet-b5',num_classes=2)
            state_dict_path = './'+strategy+'_Anim_Finetune_Ver3/CV'+str(cv)+'/'+str(epoch)+'.h5'
            test_model.load_state_dict(torch.load(state_dict_path,map_location='cuda:0'))
            '''Step 3: Test'''
            labels,preds = test(metadata=meta,col_name=strategy,model=test_model,dataloaders=dataloaders_dict,device=device)
            results[strategy][epoch]['labels']=results[strategy][epoch]['labels'] + labels
            results[strategy][epoch]['preds'] = results[strategy][epoch]['preds'] + preds
        target_names = ['Abnormal', 'Normal']        

Loaded pretrained weights for efficientnet-b5
Loaded pretrained weights for efficientnet-b5
Loaded pretrained weights for efficientnet-b5
Loaded pretrained weights for efficientnet-b5
Loaded pretrained weights for efficientnet-b5


In [9]:
MURA_report = classification_report(y_true=results['MURA'][epoch]['labels'],
                                    y_pred=results['MURA'][epoch]['preds'],
                                    target_names = target_names)
print(f"\n\nCurrent strategy\t{strategy}\tcurrent epoch\t{str(epoch)}")
print(MURA_report)



Current strategy	MURA	current epoch	99
              precision    recall  f1-score   support

    Abnormal       0.83      0.85      0.84       356
      Normal       0.61      0.57      0.59       144

    accuracy                           0.77       500
   macro avg       0.72      0.71      0.71       500
weighted avg       0.77      0.77      0.77       500



In [10]:
# Direct_report = classification_report(y_true=results['Direct'][99]['labels'],
#                                               y_pred=results['Direct'][99]['preds'],
#                                               target_names=target_names)
# print(Direct_report)

In [11]:
meta

,Index,Patient No.,Image No.,Species,Thoracic or Pelvic,Is_Normal,Image Finding,Direct Correct,Direct_Abn_Score,Direct_Nor_Score,MURA Correct,MURA_Abn_Score,MURA_Nor_Score
0,1,1,001,Canine,Thoracic,Abnormal,Osteochondrosis,1,1.000,0.000,1,0.998,0.002
1,2,1,002,Canine,Thoracic,Abnormal,Osteochondrosis,1,0.651,0.349,1,0.975,0.025
2,3,1,003,Canine,Thoracic,Abnormal,Osteochondrosis,0,0.329,0.671,0,0.349,0.651
3,4,1,004,Canine,Thoracic,Abnormal,Osteochondrosis,1,0.991,0.009,0,0.221,0.779
4,5,1,005,Canine,Thoracic,Abnormal,Osteochondrosis,1,1.000,0.000,1,1.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,140,496,Feline,Thoracic,Abnormal,Mixed Lesion,1,1.000,0.000,1,0.974,0.026
496,497,140,497,Feline,Thoracic,Abnormal,Mixed Lesion,0,0.095,0.905,1,0.987,0.013
497,498,141,498,Feline,Pelvic,Normal,Normal,0,0.513,0.487,1,0.093,0.907
498,499,141,499,Feline,Pelvic,Abnormal,DJD,0,0.013,0.987,0,0.066,0.934


In [13]:
meta.to_csv('./To be added with class score.csv',index=None,encoding='utf-8')